In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession

In [51]:
spark=SparkSession.builder\
    .appName("Olympics")\
    .getOrCreate()

In [52]:
rdd=sc.textFile('c:/users/abhishek/Downloads/olympics_data.csv')

In [53]:
rdd.take(2)

['Michael Phelps\t23\tUnited States\t2008\t08-24-08\tSwimming\t8\t0\t0\t8',
 'Michael Phelps\t19\tUnited States\t2004\t08-29-04\tSwimming\t6\t0\t2\t8']

In [54]:
rdd=rdd.map(lambda line: line.split('\t'))

In [55]:
rdd.take(2)

[['Michael Phelps',
  '23',
  'United States',
  '2008',
  '08-24-08',
  'Swimming',
  '8',
  '0',
  '0',
  '8'],
 ['Michael Phelps',
  '19',
  'United States',
  '2004',
  '08-29-04',
  'Swimming',
  '6',
  '0',
  '2',
  '8']]

In [56]:
from pyspark.sql import Row

In [57]:
df=rdd.map(lambda line:Row(Athelete=line[0],
                           Age=line[1],
                           Country=line[2],
                           Year=line[3],
                           ClosingDate=line[4],
                           Sport=line[5],
                           GoldMedals=line[6],
                           SilverMedals=line[7],
                           BronzeMedals=line[8],
                           TotalMedals=line[9])).toDF()

In [58]:
df.show()

+---+--------------------+------------+-----------+-------------+----------+------------+--------------------+-----------+----+
|Age|            Athelete|BronzeMedals|ClosingDate|      Country|GoldMedals|SilverMedals|               Sport|TotalMedals|Year|
+---+--------------------+------------+-----------+-------------+----------+------------+--------------------+-----------+----+
| 23|      Michael Phelps|           0|   08-24-08|United States|         8|           0|            Swimming|          8|2008|
| 19|      Michael Phelps|           2|   08-29-04|United States|         6|           0|            Swimming|          8|2004|
| 27|      Michael Phelps|           0|   08-12-12|United States|         4|           2|            Swimming|          6|2012|
| 25|    Natalie Coughlin|           3|   08-24-08|United States|         1|           2|            Swimming|          6|2008|
| 24|       Aleksey Nemov|           3|   10-01-00|       Russia|         2|           1|          Gymna

In [65]:
df.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Athelete: string (nullable = true)
 |-- BronzeMedals: string (nullable = true)
 |-- ClosingDate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- GoldMedals: string (nullable = true)
 |-- SilverMedals: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- TotalMedals: string (nullable = true)
 |-- Year: string (nullable = true)



In [63]:
from pyspark.sql.types import *

In [68]:
def changeType(columnName):
    global df
    df=df.withColumn(columnName,df[columnName].cast(IntegerType()))

changeType("Age")
changeType("BronzeMedals")
changeType("SilverMedals")
changeType("GoldMedals")
changeType("TotalMedals")
changeType("Year")

In [69]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Athelete: string (nullable = true)
 |-- BronzeMedals: integer (nullable = true)
 |-- ClosingDate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- GoldMedals: integer (nullable = true)
 |-- SilverMedals: integer (nullable = true)
 |-- Sport: string (nullable = true)
 |-- TotalMedals: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [87]:
df.createOrReplaceTempView('table')

In [119]:
#Top 10 countries that won the max total medals
query="SELECT Country,SUM(TotalMedals) AS Medals FROM table GROUP BY Country ORDER BY Medals DESC"
Top_medals=spark.sql(query)
Top_medals.show(10)

+-------------+------+
|      Country|Medals|
+-------------+------+
|United States|  1312|
|       Russia|   768|
|      Germany|   629|
|    Australia|   609|
|        China|   530|
|       Canada|   370|
|        Italy|   331|
|Great Britain|   322|
|  Netherlands|   318|
|       France|   318|
+-------------+------+
only showing top 10 rows



In [120]:
#Number of medals India won Year Wise
query="SELECT Year,SUM(TotalMedals) FROM table WHERE Country='India' GROUP BY Year ORDER BY Year"
IN_medals=spark.sql(query)
IN_medals.show()

+----+----------------+
|Year|sum(TotalMedals)|
+----+----------------+
|2000|               1|
|2004|               1|
|2008|               3|
|2012|               6|
+----+----------------+



In [126]:
#Medals won by India according to sports
query="SELECT Sport,SUM(TotalMedals) AS Medals FROM table WHERE Country='India' GROUP BY Sport ORDER BY Medals DESC"
IN_Sports=spark.sql(query)
IN_Sports.show()

+-------------+------+
|        Sport|Medals|
+-------------+------+
|     Shooting|     4|
|    Wrestling|     3|
|       Boxing|     2|
|    Badminton|     1|
|Weightlifting|     1|
+-------------+------+



In [140]:
#Which athelete won the most number of medals
rdd.reduce(lambda x,y:x if x[9]>y[9] else y)[0]

'Michael Phelps'

In [141]:
#The above problem can also be done as below
query="SELECT DISTINCT Athelete FROM table WHERE TotalMedals=(SELECT MAX(TotalMedals) FROM table)"
BestAth=spark.sql(query)
BestAth.show()

+--------------+
|      Athelete|
+--------------+
|Michael Phelps|
+--------------+



In [145]:
#Medals won by Usain Bolt
df.filter(df["Athelete"]=="Usain Bolt").show()

+---+----------+------------+-----------+-------+----------+------------+---------+-----------+----+
|Age|  Athelete|BronzeMedals|ClosingDate|Country|GoldMedals|SilverMedals|    Sport|TotalMedals|Year|
+---+----------+------------+-----------+-------+----------+------------+---------+-----------+----+
| 25|Usain Bolt|           0|   08-12-12|Jamaica|         3|           0|Athletics|          3|2012|
| 21|Usain Bolt|           0|   08-24-08|Jamaica|         3|           0|Athletics|          3|2008|
+---+----------+------------+-----------+-------+----------+------------+---------+-----------+----+

